In [44]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd
# 展示操作一个用户的时候应该如何使用
mongomus = mus.UserSongRecom(user_id="1a78ac10b67911ed95dd00155dadb10b")
# mongomus.makeRecomUsersSetAnswer(limit="ONE")

In [45]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "RecomUsers": 1}
doc = mongomus.rdb.findDocument(
    collection_name="recom", query=mongomus.query, projection=projections)

# 储存请求用户的名字
mongomus.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['RecomUsers'])
# 构成xy轴的内容源头
docs = mongomus.getSongsWithUserIdFromMongo(array=list_users_id)
# print(docs)

In [55]:
for item in doc[0].keys():
    print(item,":")
    print(doc[0][item])

id :
1a78ac10b67911ed95dd00155dadb10b
name :
RuRuUIH
RecomUsers :
[]


In [47]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus.makeXYMatric(docs=docs)
# print(data)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus.y, users=mongomus.x)
    

In [48]:
sudf.df

,RuRuUIH
35504129,1
29100034,1
19918849,1
16435207,1
5027852,1
...,...
468738013,1
32309218,1
409102312,1
410181618,1


In [49]:
# 推荐过程
# dict_user_song = mongomus.makeRecomDcition(df_object=sudf, sign=-1)
similar = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [50]:
similar

,RuRuUIH
RuRuUIH,1.0


In [51]:
# 使用平均数，对以上的相似度结果求平均，并取高于平均数的对象形成一个list
diction_user = sudf.makeTopNUsers(similar=similar,sign=1)
print(diction_user)
diction = sudf.makeRecomUserBySong(
            topN=diction_user, song_num=50)

{'RuRuUIH': []}


In [52]:
# 程序过程中会有sign 来判断是去单个，还是返回所有的
dict_result = {mongomus.user_name: [diction[mongomus.user_name],diction_user[mongomus.user_name]]}
print(dict_result)

{'RuRuUIH': [[], []]}


In [53]:
print(dict_result)

{'RuRuUIH': [[], []]}


In [54]:
# 最后保存
# mongomus.saveUserSongRecomAnswer(diction=dict_result)
docs = mongomus.changeDataFormat(diction=dict_result, list_keys=list(dict_result))
# print(docs)
mongomus.wdb.writeDocument(
    docs=docs, collection_name="recom")


update coll RuRuUIH in recom
